**Mobile Apps Analytics**

*This project helps the organization to understand the popular genre and reach of the mobile apps which in turn helps them to decide on what to do next to maximize the revenue.*

** Let us read the mobile apps raw data and store as list **

In [2]:
from csv import reader
ios_apps_list =[]
gp_apps_list =[]
open_ios_file = open ('AppleStore.csv',encoding='utf8') # open the file containing ios apps data
open_gp_file = open ('googleplaystore.csv', encoding='utf8') 
ios_file_reader = reader(open_ios_file) # read the ios apps file
gp_file_reader = reader(open_gp_file) 
ios_apps_list = list(ios_file_reader) # store the ios app data as list
gp_apps_list = list(gp_file_reader) 

**This function helps you to play around a dataset**

In [ ]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

**Let us explore our mobile apps data by using the above function**

In [ ]:
print('sample ios apps')
explore_data(ios_apps_list,1,3)
print('sample google play apps')
explore_data(gp_apps_list,1,3)

***oops! While exploring we find out a inconsistant row,Let us kick out !***

In [3]:
del gp_apps_list[10473]

*** Let us look out for duplicate data in our dataset to have more acuuracy in our analysis***

In [ ]:
unique_gp_apps = []
dup_gp_apps = []
# Find duplicate entries of apps
for apps in gp_apps_list[1:] :
    apps_name = apps[0]
    if apps_name in unique_gp_apps :
        dup_gp_apps.append(apps_name)
    else :
        unique_gp_apps.append(apps_name)
        
# print unique app names
print(unique_gp_apps[:10])
print('-----dup apps-----')
# print dup app names
print(dup_gp_apps[:10])

# duplicate apps count

print('No of duplicate apps :')

print(len(dup_gp_apps))

***The duplicates should be removed based on the relationship differences between all rows rather than simply removing by app name or other key column. For instance, removing duplicate by version and no of reviews***

In [ ]:
# Dicitionary of app names & review count

reviews_max = {}

for app in gp_apps_list[1:]:
    name=app[0]
    n_reviews=float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name]=n_reviews
    elif name not in reviews_max:
        reviews_max[name]=n_reviews
        
print(len(reviews_max))

android_clean = []
already_added = []

for app in gp_apps_list[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
    else:
        already_added.append(name)

print(android_clean[:2])

    

*** We have strategized to only english speak market. So, let's built a function to look for non english characters ***

In [ ]:
def is_eng_str(str):
    char_cnt = 0
    for char in str:
        if ord(char) > 127:
            char_cnt+=1
    if char_cnt > 3:
        return False
    else:
        return True

print(is_eng_str('爱奇艺PPS -《欢乐颂2》电视剧热播'))

*** let's filter out non-english apps!!! ***

In [ ]:
# Filter out non english apps
eng_apps =[]
non_eng_apps =[]
for apps in gp_apps_list[1:] :
    apps_name = apps[0]
    if is_eng_str(apps_name) == True:
        eng_apps.append(apps_name)
    elif is_eng_str(apps_name) == False:
        non_eng_apps.append(apps_name)
        
print(eng_apps[:10])
print(non_eng_apps[:10])

***As said earlier,We are going to built out only free apps and generate revenue through in-ads revenue. So, let's filter only free apps***

In [ ]:

# Isolate free apps 

def isolate_apps(apps_data,dice_idx):
    free_apps =[]
    paid_apps = []
    for app in apps_data[1:]:
        if app[dice_idx]=='0' or app[dice_idx]=='0.0':
            free_apps.append(app)
        else:
            paid_apps.append(app)
    return free_apps,paid_apps

gp_free_apps,gp_paid_apps=isolate_apps(gp_apps_list,7)
ios_free_apps,ios_paid_apps=isolate_apps(ios_apps_list,4)
print('no of google play free apps:')
print(len(gp_free_apps))
print('no of ios free apps:')
print(len(ios_free_apps))
print('------google play free apps-----')
print(gp_free_apps[:2])
print('------google play paid apps-----')
print(gp_paid_apps[:2])
print('------ios play free apps-----')
print(ios_free_apps[:2])
print('------ios play paid apps-----')
print(ios_paid_apps[:2])
    
    

*** Our main goal is to build an app which yeilds maximum revenue. To achieve this, let's analyse both data set and find the most the attracted mobile app by user based on various dimensions***

In [5]:
'''
Desc : A function to create a frequency table from a dataset(list of lists)
       and display the dicitionary values in descending order.

Param_1: dataset 

Param_2: index of the column

Returns : A list of tuples in descending order.

'''

def freq_table(dataset,index):
    col_values=[]
    res_dict = {}
    for row in dataset[1:]:
        value=row[index]
        col_values.append(value)
    for key in col_values:
        if key in res_dict:
            res_dict[key]+=1
        else:
            res_dict[key]=1
    
    tot_no_of_keys = len(col_values)  
    for key in res_dict:
        res_dict[key]=round((res_dict[key]/tot_no_of_keys)*100,2)

    return res_dict


def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

print('--------ios_app_genre------------')
display_table(ios_apps_list,11)
print('--------gp_apps_category---------')
display_table(gp_apps_list,1)
print('--------gp_apps_genre------------')
display_table(gp_apps_list,9)

--------ios_app_genre------------
Games : 53.66
Entertainment : 7.43
Education : 6.29
Photo & Video : 4.85
Utilities : 3.45
Health & Fitness : 2.5
Productivity : 2.47
Social Networking : 2.32
Lifestyle : 2.0
Music : 1.92
Shopping : 1.7
Sports : 1.58
Book : 1.56
Finance : 1.45
Travel : 1.13
News : 1.04
Weather : 1.0
Reference : 0.89
Food & Drink : 0.88
Business : 0.79
Navigation : 0.64
Medical : 0.32
Catalogs : 0.14
--------gp_apps_category---------
FAMILY : 18.19
GAME : 10.55
TOOLS : 7.78
MEDICAL : 4.27
BUSINESS : 4.24
PRODUCTIVITY : 3.91
PERSONALIZATION : 3.62
COMMUNICATION : 3.57
SPORTS : 3.54
LIFESTYLE : 3.52
FINANCE : 3.38
HEALTH_AND_FITNESS : 3.15
PHOTOGRAPHY : 3.09
SOCIAL : 2.72
NEWS_AND_MAGAZINES : 2.61
SHOPPING : 2.4
TRAVEL_AND_LOCAL : 2.38
DATING : 2.16
BOOKS_AND_REFERENCE : 2.13
VIDEO_PLAYERS : 1.61
EDUCATION : 1.44
ENTERTAINMENT : 1.37
MAPS_AND_NAVIGATION : 1.26
FOOD_AND_DRINK : 1.17
HOUSE_AND_HOME : 0.81
LIBRARIES_AND_DEMO : 0.78
AUTO_AND_VEHICLES : 0.78
WEATHER : 0.76
ART_

*** From the frequency table of ios mobile applications, we could see the most common genre is "Games" which shares more than half of the industry, 53.66% and the runner up is entertainment genre which shares 7.43%. Just by going through the analysis, most of the apps are designed for games and entertainment purpose. I would recommend to build out an mobile application in entertainment genre because revenue generation opporunities are more in social networking apps than games because of large user base. ***

*** From the frequency table of google play applications, we could see the most common genres are "Family" and "Tools". And Entertainment genre also tops the list. By comparing both the analysis of ios mobile applications and google play applications, "Entertainment" genre is the suitable genre to take up and roll out a mobile application***

In [15]:
# Calculate average no of installs in ios_apps

ios_genre_agg = freq_table(ios_apps_list,11)
for genre in ios_genre_agg:
    total=0
    len_genre=0
    for row in ios_apps_list[1:]:
        genre_app=row[11]
        no_of_usr_ratings=float(row[5])
        if genre == genre_app:
            total+=no_of_usr_ratings
            len_genre+=1
    avg_usr_ratings = total/len_genre
    print(genre,':',avg_usr_ratings)  



Reference : 22410.84375
Education : 2239.2295805739514
News : 13015.066666666668
Book : 5125.4375
Health & Fitness : 9913.172222222222
Medical : 592.7826086956521
Navigation : 11853.95652173913
Shopping : 18615.32786885246
Travel : 14129.444444444445
Productivity : 8051.3258426966295
Social Networking : 45498.89820359281
Food & Drink : 13938.619047619048
Music : 28842.021739130436
Photo & Video : 14352.280802292264
Sports : 14026.929824561403
Weather : 22181.027777777777
Business : 4788.087719298245
Lifestyle : 6161.763888888889
Utilities : 6863.822580645161
Finance : 11047.653846153846
Games : 13691.996633868463
Entertainment : 7533.678504672897
Catalogs : 1732.5


*** By looking into the frequency table,I would recommend to build an mobile applications on a "Entertainment" genre.***

In [16]:
# To create to frequency table for android apps based on category
gp_apps_ctgry=freq_table(gp_apps_list,1)
category_installs={}
for ctgry in gp_apps_ctgry:
    total=0
    len_category=0
    for row in gp_apps_list[1:]:
        category=row[1]
        installs=row[5]
        installs_tfd=installs.replace('+','')
        installs_tfd=float(installs_tfd.replace(',',''))
        if category==ctrgy:
            total+=installs_tfd
            len_category+=1
    avg_installs=total/len_category
    print(ctgry,':',avg_installs)

            

NameError: name 'ctrgy' is not defined